In [ ]:
import ee
import geemap

# load map
Map = geemap.Map(center=[40,-100], zoom=4)


In [ ]:
# 1.Load ROI
roi = ee.FeatureCollection('your id/assets name/ shapefile name')

# 2.Define Mask function
def mask_edge(image):
  edge = image.lt(-30.0)
  masked_image = image.mask().And(edge.Not())
  return image.updateMask(masked_image)

def ascOrdesc(dataset, date, roi):
  desc = dataset.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
  asc = dataset.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))
  asc = asc.filterBounds(roi.geometry()).filter(date).mean()
  desc = desc.filterBounds(roi.geometry()).filter(date).mean()
  return asc,desc

# 3.Aquire Sentinel-1 dataset
dataset_vv = (
    ee.ImageCollection('COPERNICUS/S1_GRD')
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
    .select('VV')
    .map(mask_edge)
)

dataset_hh = (
    ee.ImageCollection('COPERNICUS/S1_GRD')
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HH'))
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
    .select('HH')
    .map(mask_edge)
)

dataset_hv = (
    ee.ImageCollection('COPERNICUS/S1_GRD')
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HV'))
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
    .select('HV')
    .map(mask_edge)
)

dataset_vh = (
    ee.ImageCollection('COPERNICUS/S1_GRD')
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
    .select('VH')
    .map(mask_edge)
)

# 4.Filter startDate and endDate
startDate = '2021-9-15'
endDate = '2021-10-15'
date = ee.Filter.date(startDate,endDate)

# 5.'DESCENDING' or 'ASCENDING'
asc_vv,desc_vv = ascOrdesc(dataset_vv,date, roi)
asc_vh,desc_vh = ascOrdesc(dataset_vh,date, roi)
asc_hh,desc_hh = ascOrdesc(dataset_hh,date, roi)
asc_hv,desc_hv = ascOrdesc(dataset_hv,date, roi)

# 6. Stack bands you need
image = ee.Image.cat(asc_vv,asc_vh)

# 7. show result
Map.centerObject(roi,8)
Map.add_layer(image, {'min': -25,'max': 5}, 'dual-polar SAR', True)

Map

In [ ]:
from tqdm import tqdm

# 定义子图导出函数
def export_subimage_drive(image, roi, output_file_name, output_folder=None, scale=None, file_format=None):

  # 生成子区域列表
  values_list = roi.aggregate_array('id').getInfo()

  # 使用 tqdm 包装循环以显示进度
  for index, value in tqdm(enumerate(values_list), total=len(values_list)):

    # 生成子区域矢量图
    test_roi = roi.filterMetadata(name='id', operator='equals', value=value)
    out_img = image.clip(test_roi)
    ### These code is used to linear strech SAR image and download by Uint8
    range_info = out_img.reduceRegion(reducer=ee.Reducer.minMax(), geometry=test_roi, scale=10)
    min_vv = range_info.get('VV' + '_min').getInfo()
    min_vh = range_info.get('VH' + '_min').getInfo()

    max_vv = range_info.get('VV' + '_max').getInfo()
    max_vh = range_info.get('VH' + '_max').getInfo()

    out_img_vv = out_img.select('VV').subtract(min_vv).divide(max_vv - min_vv).multiply(255).round().toUint8()
    out_img_vh = out_img.select('VH').subtract(min_vh).divide(max_vh - min_vh).multiply(255).round().toUint8()

    out_img = ee.Image.cat(out_img_vv, out_img_vh)
    ###

    # 自定义更多导出参数
    region = test_roi  # 导出的区域
    file_format = file_format  # 导出的文件格式

    # 设置配置参数用于输出图像
    task_config = {
        'folder': output_folder,  # 输出到 Google Drive 文件夹
        'region': region.geometry(),  # ROI
        'scale': scale,  # 图像分辨率
        'crs': 'EPSG:4326',
        'maxPixels': 1.0E13,
        'fileFormat': file_format
    }
    description = f'{output_file_name + "_" + str(value)}'

    # 输出图像到 Google Drive
    task = ee.batch.Export.image.toDrive(out_img, description, **task_config)
    task.start()
    # print("Exporting {}".format(description))


In [ ]:
# export Image
export_subimage_drive(image,roi,output_file_name='file_name',output_folder="folder_name",scale=10, file_format='GEO_TIFF')